# Machine learning and River Morphology

Understand how to use Suport Vector Machine to classify river morphology

*Image here showing river bancks*
    



## Import packages and load data

In [37]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine # example dataset for wine classification
from sklearn.model_selection import train_test_split # function to split testing and training datasets
from sklearn.svm import SVC # class Support Vector Machine
from sklearn.model_selection import validation_curve # function too stract error through validation
from sklearn.metrics import plot_confusion_matrix #function to plot confution matrix
import matplotlib.pyplot as plt


data= pd.read_excel("bedload-recking-sample.xlsx")
data.drop(columns=["ID",data.columns[0],"Date", "D16", "D50", "D84", "D90","R"], inplace=True)
data


,W,S,Q,U,H,qs,Morphology
0,7.850000,0.0160,1.90000,0.85,0.29,0.000614,Riffle-pool
1,6.190000,0.0202,0.93000,0.49,0.31,0.000109,Plane Bed
2,24.015192,0.0030,5.71607,NaN,NaN,0.000281,NaN
3,56.390000,0.0038,176.13000,2.05,1.52,0.025500,Riffle-pool
4,12.820000,0.0019,9.78000,NaN,0.86,0.001200,Riffle-pool
...,...,...,...,...,...,...,...
995,7.890000,0.0160,5.16000,1.47,0.46,0.010600,Plane Bed
996,3.500000,0.0096,2.70000,NaN,0.36,3.750000,Riffle-pool
997,3.000000,0.0570,0.31000,0.73,NaN,0.008330,Step-pool
998,25.610000,0.0014,22.17000,1.20,NaN,0.059700,Riffle-pool


# Clean data
### Remove NaN values

In [2]:
# verify the type of data within the columns
print(data.dtypes,"\n")

 
# check for NaN under the columns
for column in data.columns.to_list():
    print(column,":",data[column].isnull().any())
    
# remove all lines that contain NaN value
print(data.dropna(inplace=True))

# verify that all NaN values are out
for column in data.columns.to_list():
    print(column,":",data[column].isnull().any())
    
# verify size of remannign data
print(data.shape)

W             float64
S             float64
Q             float64
U             float64
H             float64
qs            float64
Morphology     object
dtype: object 

W : True
S : False
Q : True
U : True
H : True
qs : False
Morphology : True
None
W : False
S : False
Q : False
U : False
H : False
qs : False
Morphology : False
(631, 7)


### Search for outliers

In [3]:
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import numpy as np

host = host_subplot(111, axes_class=AA.Axes)
plt.subplots_adjust(right=0.75)

labels = data.columns[0:6].tolist()
colors = ["crimson", "purple", "limegreen", "gold", "blue", "gray"]

width=0.5

std_store = []
mean_store = []
for i, l in enumerate(labels):
    if i ==0:
        ax = host
        ax.set_ylabel(labels[i])
    else:        
        ax = host.twinx()
        new_fixed_axis = ax.get_grid_helper().new_fixed_axis
        ax.axis["right"] = new_fixed_axis(loc="right",
                                            axes=ax,
                                            offset=(60*(i-1), 0))
        ax.axis["right"].toggle(all=True)
        ax.set_ylabel(labels[i])

    x = np.ones(data.shape[0])*i + (np.random.rand(data.shape[0])*width-width/2.)
    ax.scatter(x, data[data.columns[i]],color=colors[i])
    mean = data[data.columns[i]].mean()
    std = np.std(data[data.columns[i]])
    ax.plot([i-width/2., i+width/2.],[mean,mean], color="k")
    ax.plot([i,i], [mean-2*std, mean+2*std], color="k")    
        


ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels)

plt.draw()
# plt.show()

### Remove outliers

+ the outliers can be identifyed by the expecialist.
+ if it is not possible to have a unbias opinion from the expecialist about the outliers, zscore can be used.

In [4]:
from scipy.stats import zscore
remove_method = "expert_analysis"

if remove_method == "expert_analysis":
    data = data.loc[(data["qs"]<0.02) &
                    (data["Q"]<2000) &
                    (data["W"]<250)]
else:
    data = data[(np.abs(zscore(data.loc[:,data.columns!="Morphology"])) < 3).all(axis=1)]
    
data

,W,S,Q,U,H,qs,Morphology
0,7.85,0.0160,1.90,0.85,0.29,0.000614,Riffle-pool
1,6.19,0.0202,0.93,0.49,0.31,0.000109,Plane Bed
5,5.33,0.0300,1.64,0.81,0.38,0.000970,Plane Bed
7,6.80,0.0200,3.79,1.33,0.42,0.008500,Step-pool
9,12.45,0.0140,2.92,0.74,0.30,0.000100,Riffle-pool
...,...,...,...,...,...,...,...
990,7.48,0.0130,4.07,1.35,0.39,0.017200,Plane Bed
991,10.97,0.0268,1.72,0.42,0.37,0.000139,Plane Bed
992,4.20,0.0450,0.94,0.26,0.29,0.002100,Plane Bed
994,12.76,0.0091,18.92,1.81,0.82,0.009420,Plane Bed


### Verify classes proportions
+ It is importanat to have a representatice number of samples for each class
+ the minimun number of sampels for each class depends on the type of problem. In general terms, the more unique are the characteristics of a class, the less samples are needed. 
+ Creating model that does not contamplate all the classes of a system will be limited of classifing samples within the limits of classes given. Thus, removing classes from the training procces is not recomended. Ideally, all classes have a large number of traning samples and they are similar in quantity. 

In [5]:
data["Morphology"].value_counts()

Plane Bed      310
Riffle-pool     91
Step-pool       71
Braiding        27
Sand bed         1
Name: Morphology, dtype: int64

### Remove under-representated class

In [6]:
data = data.loc[(data["Morphology"] != "Sand bed")&
               (data["Morphology"] != "Braiding")
               ]
data 

,W,S,Q,U,H,qs,Morphology
0,7.85,0.0160,1.90,0.85,0.29,0.000614,Riffle-pool
1,6.19,0.0202,0.93,0.49,0.31,0.000109,Plane Bed
5,5.33,0.0300,1.64,0.81,0.38,0.000970,Plane Bed
7,6.80,0.0200,3.79,1.33,0.42,0.008500,Step-pool
9,12.45,0.0140,2.92,0.74,0.30,0.000100,Riffle-pool
...,...,...,...,...,...,...,...
990,7.48,0.0130,4.07,1.35,0.39,0.017200,Plane Bed
991,10.97,0.0268,1.72,0.42,0.37,0.000139,Plane Bed
992,4.20,0.0450,0.94,0.26,0.29,0.002100,Plane Bed
994,12.76,0.0091,18.92,1.81,0.82,0.009420,Plane Bed


# Derive new predictors

+ Sometimes it is possible to derive new meaniful predictors from the existing ones.
+ in the example: 
 + the product of slope S (-), velocity U (m/s), and Depth H  (m)
 + the ratio of discharge (m³/s) and width (m) (i.e. Q/W)
+ rise the problem of correlated predictors
+ when it is ok to keep coorelated predictors


In [7]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

data["SUH"] = (data.loc[:,"S"]*data.loc[:,"U"]*data.loc[:,"H"])
data["Q/W"] = (data.loc[:,"Q"]/data.loc[:,"W"])
data

,W,S,Q,U,H,qs,Morphology,SUH,Q/W
0,7.85,0.0160,1.90,0.85,0.29,0.000614,Riffle-pool,0.003944,0.242038
1,6.19,0.0202,0.93,0.49,0.31,0.000109,Plane Bed,0.003068,0.150242
5,5.33,0.0300,1.64,0.81,0.38,0.000970,Plane Bed,0.009234,0.307692
7,6.80,0.0200,3.79,1.33,0.42,0.008500,Step-pool,0.011172,0.557353
9,12.45,0.0140,2.92,0.74,0.30,0.000100,Riffle-pool,0.003108,0.234538
...,...,...,...,...,...,...,...,...,...
990,7.48,0.0130,4.07,1.35,0.39,0.017200,Plane Bed,0.006844,0.544118
991,10.97,0.0268,1.72,0.42,0.37,0.000139,Plane Bed,0.004165,0.156791
992,4.20,0.0450,0.94,0.26,0.29,0.002100,Plane Bed,0.003393,0.223810
994,12.76,0.0091,18.92,1.81,0.82,0.009420,Plane Bed,0.013506,1.482759


### Verify colinearity
+ for effect of prediciton, multicollinearyty may not be a problem. However, variable importance analysis is not reliable whe correlated feactures are used. The weight of a model may not represent correctly the importance of the variable (verify this statment)
+ One tool that ven measure level of correlation can be the variance correlation factor.
+ Multicollinearity occurs when there are two or more independent variables in a multiple regression model, which have a high correlation among themselves. When some features are highly correlated, we might have difficulty in distinguishing between their individual effects on the dependent variable. Multicollinearity can be detected using various techniques, one such technique being the Variance Inflation Factor(VIF).
+ What is the limit for VIF?
+ Explain how id VIF computed. 

In [8]:
# Analyse viasually the predictors that may have high VIF.

import seaborn as sns
sns.pairplot(data.loc[:,data.columns!="Morphology"])
# plt.show()

In [9]:
# Analyse coorralation with pandas
data.loc[:,data.columns!="Morphology"].corr()

,W,S,Q,U,H,qs,SUH,Q/W
W,1.000000,-0.428163,0.880958,0.450838,0.910303,0.142708,-0.117569,0.894620
S,-0.428163,1.000000,-0.248648,-0.374155,-0.415567,-0.176459,0.365151,-0.364324
Q,0.880958,-0.248648,1.000000,0.397097,0.899995,0.161044,-0.056120,0.936792
U,0.450838,-0.374155,0.397097,1.000000,0.500770,0.450743,0.503766,0.602042
H,0.910303,-0.415567,0.899995,0.500770,1.000000,0.196424,-0.018437,0.963946
qs,0.142708,-0.176459,0.161044,0.450743,0.196424,1.000000,0.187472,0.241445
SUH,-0.117569,0.365151,-0.056120,0.503766,-0.018437,0.187472,1.000000,0.062000
Q/W,0.894620,-0.364324,0.936792,0.602042,0.963946,0.241445,0.062000,1.000000


In [10]:
from sklearn.linear_model import LinearRegression

# compute VIF for all the predictors
def calculate_vif(df, features):
    vif, tolerance = {}, {}
    # all the features that you want to examine
    for feature in features:
        # extract all the other features you will regress against
        X = [f for f in features if f != feature]
        X, y = df[X], df[feature]
        # extract r-squared from the fit
        r2 = LinearRegression().fit(X, y).score(X, y)

        # calculate tolerance
        tolerance[feature] = 1 - r2
        # calculate VIF
        vif[feature] = 1 / (tolerance[feature])
    # return VIF DataFrame
    return pd.DataFrame({'VIF': vif, 'Tolerance': tolerance})

calculate_vif(df=data, 
              features=data.columns[data.columns != "Morphology"].to_list())




,VIF,Tolerance
W,7.665465,0.130455
S,2.762193,0.362031
Q,16.728897,0.059777
U,6.730446,0.148579
H,25.802867,0.038755
qs,1.298133,0.770337
SUH,3.163537,0.316102
Q/W,55.177649,0.018123


In [11]:
# remove predictors that have High VIF
# data = data.loc[:, data.columns[(data.columns !="H")&
#                                 (data.columns !="Q")
#                                ]
#                ]

calculate_vif(df=data, 
              features=data.columns[data.columns != "Morphology"].to_list())


,VIF,Tolerance
W,7.665465,0.130455
S,2.762193,0.362031
Q,16.728897,0.059777
U,6.730446,0.148579
H,25.802867,0.038755
qs,1.298133,0.770337
SUH,3.163537,0.316102
Q/W,55.177649,0.018123


+ Q is alredy a variable insie Q/W and H will be "represented" Q/W
+ Why it make sense remove H?

# Split training and testing sets

In [12]:
labels = data.loc[:, "Morphology"]
predictors = data.loc[:, data.columns[data.columns != "Morphology"]]


X_train, X_test, y_train, y_test = train_test_split(predictors,  
                                                    labels,
                                                    test_size=0.3, # percentage of data that goes into the testing set
                                                    random_state=42 # seed for random selection of data
                                                   )

# visualize
print("TRAINING DATASET PREDICTORS")
print(X_train.head(), "\n")
print("dataframe size", X_train.shape,"\n")
print("------------------------------------------")
print("TRAINING DATASET TARGET")
print(y_train,"\n")
print()
print("------------------------------------------------------------------------------------\n")

print("TESTING DATASET PREDICTORS")
print(X_test.head())
print("dataframe size", X_test.shape,"\n")
print("------------------------------------------")
print("TESTING DATASET TARGET")
print(y_test.head())
print("Vector size", y_test.shape)


TRAINING DATASET PREDICTORS
         W       S       Q     U     H        qs       SUH       Q/W
633   8.35  0.0210    3.56  1.22  0.37  0.007600  0.009479  0.426347
341  84.73  0.0021  203.03  1.47  1.63  0.000053  0.005032  2.396200
61    5.24  0.0233    0.49  0.54  0.18  0.000711  0.002265  0.093511
18    8.46  0.0131    2.93  1.01  0.37  0.008200  0.004895  0.346336
918   8.23  0.0320    0.34  0.24  0.17  0.000063  0.001306  0.041312 

dataframe size (330, 8) 

------------------------------------------
TRAINING DATASET TARGET
633    Riffle-pool
341      Plane Bed
61       Plane Bed
18     Riffle-pool
918      Step-pool
          ...     
238      Plane Bed
582      Plane Bed
739      Plane Bed
935      Step-pool
225      Plane Bed
Name: Morphology, Length: 330, dtype: object 


------------------------------------------------------------------------------------

TESTING DATASET PREDICTORS
         W       S     Q     U     H        qs       SUH       Q/W
133   8.05  0.0146  0.87  

## k-fold Cross Validation

In [13]:
# interval of possible parameters 
param_range = np.logspace(-2, 3, 30) 

# Peform 5-fold cross-validation and save training and validation error
train_scores, test_scores = validation_curve(
    SVC(),
    X_train, 
    y_train,
    param_name="gamma", # parameter to vary
    param_range=param_range,
    scoring="accuracy", 
    cv=5,    # number of folders
)


# convert accuracy into error
train_error = 1-train_scores
validation_error = 1-test_scores

# compute 5-fold cross-validation mean and std of error 
train_error_mean = np.mean(train_error, axis=1)
train_error_std = np.std(train_error, axis=1)
validation_error_mean = np.mean(validation_error, axis=1)
validation_error_std = np.std(validation_error, axis=1)

# visualize
plt.title("Validation Curve with SVM")
plt.xlabel(r"$\gamma$")
plt.ylabel("Error (1-accuracy)")
plt.ylim(-0.01, 1.1)
lw = 2
plt.semilogx(
    param_range, train_error_mean, label="Training score", color="darkorange", lw=lw
)
plt.fill_between(
    param_range,
    train_error_mean - train_error_std,
    train_error_mean + train_error_std,
    alpha=0.2,
    color="darkorange",
    lw=lw,
)
plt.semilogx(
    param_range, validation_error_mean, label="Cross-validation score", color="navy", lw=lw
)
plt.fill_between(
    param_range,
    validation_error_mean - validation_error_std,
    validation_error_mean + validation_error_std,
    alpha=0.2,
    color="navy",
    lw=lw,
)
plt.legend(loc="best")
# plt.show()

## Choose optimal parameter


In [14]:
# save optimal gamma
index_min = min(range(len(validation_error_mean)), key=validation_error_mean.__getitem__)
validation_error_min = validation_error_mean[index_min]
gamma_opt = param_range[index_min] # gamma that gives minimun validation error

# visualize 
columns = ["Mean validation error","gamma"]
array = np.array([validation_error_mean, param_range]).transpose()

print( pd.DataFrame(array,columns=columns), "\n")
print("Minimun mean validation error:", validation_error_min)
print("optimal gamma:", gamma_opt)

    Mean validation error        gamma
0                0.345455     0.010000
1                0.345455     0.014874
2                0.351515     0.022122
3                0.351515     0.032903
4                0.336364     0.048939
5                0.330303     0.072790
6                0.318182     0.108264
7                0.324242     0.161026
8                0.303030     0.239503
9                0.275758     0.356225
10               0.263636     0.529832
11               0.257576     0.788046
12               0.236364     1.172102
13               0.233333     1.743329
14               0.239394     2.592944
15               0.239394     3.856620
16               0.242424     5.736153
17               0.248485     8.531679
18               0.254545    12.689610
19               0.266667    18.873918
20               0.281818    28.072162
21               0.287879    41.753189
22               0.318182    62.101694
23               0.333333    92.367086
24               0.339394

## Train the optimal hypothesis model

In [15]:
# train optimal model and evaluate accuracy 
h_opt = SVC(gamma=gamma_opt) # instiate optimal model 
h_opt.fit(X_train,y_train) # train the model to the entire training set
print("Error (1-Accuracy) of h_opt on testing data: \n -->>",1- h_opt.score(X_test,y_test)) 

Error (1-Accuracy) of h_opt on testing data: 
 -->> 0.27464788732394363


## Performance evaluation

In [16]:
#plot confution Matrix
plot_confusion_matrix(h_opt, # trained optimal hypothesis model
                      X_test, 
                      y_test,
                     )
# plt.show()

# Further improvent of the model
+ select the features that are most valiable for increasing the model classification

In [20]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif

select_feature = SelectKBest(mutual_info_classif,k=4).fit(X_train, y_train)

selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)


,Feature,Scores
1,S,0.627278
0,W,0.370178
2,Q,0.137963
5,qs,0.126672
3,U,0.094999
4,H,0.080065
7,Q/W,0.075943
6,SUH,0.038836


### remove variables with low score
+ this can increase the performance of the model

In [18]:
print(h_opt.fit(X_train,y_train).score(X_test,y_test))
print(h_opt.fit(X_train_new,y_train).score(X_test.loc[:,["W","Q/W","U","S"]],y_test))


0.7253521126760564
0.6619718309859155
